# Square Lattice from Neighbor List

In [30]:
using Combinatorics

# Define the lattice constant (distance between atoms)
lattice_constant = 1.0

# Function to generate a 2D lattice of hydrogen atoms
function generate_2d_lattice(rows, cols, lattice_constant)
    positions = Dict{Int, Tuple{Float64, Float64}}()
    index = 1
    
    for i in 1:rows
        for j in 1:cols
            x = (i - 1) * lattice_constant
            y = (j - 1) * lattice_constant
            positions[index] = (x, y)
            index += 1
        end
    end
    return positions
end

# Function to compute first and second nearest neighbors using minimum image distance
function compute_neighbors(positions, rows, cols, lattice_constant)
    neighbors = Dict{Int, Tuple{Vector{Int}, Vector{Int}}}()
    num_atoms = rows * cols
    box_length_x = rows * lattice_constant
    box_length_y = cols * lattice_constant
    half_box_length_x = box_length_x / 2.0
    half_box_length_y = box_length_y / 2.0
    second_nearest_distance = sqrt(2) * lattice_constant

    for i in 1:num_atoms
        first_neighbors = Int[]
        second_neighbors = Int[]
        pos_i = positions[i]
        
        for j in 1:num_atoms
            if i != j
                pos_j = positions[j]
                dx = pos_j[1] - pos_i[1]
                dy = pos_j[2] - pos_i[2]
                
                # Apply minimum image convention
                if dx > half_box_length_x
                    dx -= box_length_x
                elseif dx < -half_box_length_x
                    dx += box_length_x
                end
                
                if dy > half_box_length_y
                    dy -= box_length_y
                elseif dy < -half_box_length_y
                    dy += box_length_y
                end
                
                distance = sqrt(dx^2 + dy^2)
                
                if distance <= lattice_constant
                    push!(first_neighbors, j)
                elseif distance <= second_nearest_distance
                    push!(second_neighbors, j)
                end
            end
        end
        
        neighbors[i] = (first_neighbors, second_neighbors)
    end
    
    return neighbors
end

abstract type LatticeWalkers end

mutable struct Lattice2DSystem
    lattice_type::Symbol
    dimensions::Tuple{Int64, Int64}
    num_occ_sites::Int64
    site_occupancy::Matrix{Bool}
    positions::Dict{Int, Tuple{Float64, Float64}}
    neighbors::Dict{Int, Tuple{Vector{Int}, Vector{Int}}}

    function Lattice2DSystem(lattice_type::Symbol, site_occupancy::Matrix{Bool})
        dims = size(site_occupancy)
        num_occ_sites = sum(site_occupancy)
        positions = generate_2d_lattice(dims[1], dims[2], lattice_constant)
        neighbors = compute_neighbors(positions, dims[1], dims[2], lattice_constant)
        return new(lattice_type, dims, num_occ_sites, site_occupancy, positions, neighbors)
    end

    function Lattice2DSystem(lattice_type::Symbol, dims::Tuple{Int64, Int64}, num_occ_sites::Int64, seed::Int64)
        total_sites = prod(dims)
        occupancy = vcat(fill(true, num_occ_sites), fill(false, total_sites - num_occ_sites))
        Random.seed!(seed)
        shuffle!(occupancy)
        site_occupancy = reshape(occupancy, dims)
        positions = generate_2d_lattice(dims[1], dims[2], lattice_constant)
        neighbors = compute_neighbors(positions, dims[1], dims[2], lattice_constant)
        return new(lattice_type, dims, num_occ_sites, site_occupancy, positions, neighbors)
    end
end

mutable struct Lattice2DWalker
    configuration::Lattice2DSystem
    energy::Float64
    iter::Int64
    function Lattice2DWalker(configuration::Lattice2DSystem; energy=0.0, iter=0)
        return new(configuration, energy, iter)
    end
end

function interaction_energy(at::Lattice2DSystem, nn_energy::Float64, nnn_energy::Float64, adsorption_energy::Float64)
    e_adsorption = at.num_occ_sites * adsorption_energy
    e_nn = 0.0
    e_nnn = 0.0
    
    for i in 1:at.dimensions[1]
        for j in 1:at.dimensions[2]
            index = (i - 1) * at.dimensions[2] + j
            if at.site_occupancy[i, j]
                for nn in at.neighbors[index][1]
                    if at.site_occupancy[div(nn - 1, at.dimensions[2]) + 1, mod(nn - 1, at.dimensions[2]) + 1]
                        e_nn += nn_energy / 2
                    end
                end
                for nnn in at.neighbors[index][2]
                    if at.site_occupancy[div(nnn - 1, at.dimensions[2]) + 1, mod(nnn - 1, at.dimensions[2]) + 1]
                        e_nnn += nnn_energy / 2
                    end
                end
            end
        end
    end
    
    return e_adsorption + e_nn + e_nnn
end

function exact_enumeration(L::Int64, M::Int64, N::Int64, lattice_type::Symbol, nn_energy::Float64, nnn_energy::Float64, adsorption_energy::Float64)
    sites = [(i, j) for i in 1:L for j in 1:M]
    all_configs = collect(combinations(sites, N))
    all_configs = [reshape([in((i, j), config) for i in 1:L, j in 1:M], L, M) for config in all_configs]
    lattices = [Lattice2DSystem(lattice_type, config) for config in all_configs]

    walkers = [Lattice2DWalker(lattice) for lattice in lattices]

    for walker in walkers
        e_interaction = interaction_energy(walker.configuration, nn_energy, nnn_energy, adsorption_energy)
        walker.energy = e_interaction
    end

    energies = [walker.energy for walker in walkers]
    configurations = [walker.configuration for walker in walkers]

    return energies, configurations
end

function compute_internal_energy_versus_temperature(L::Int64, N::Int64, T_min::Float64, T_max::Float64, num_points::Int64, nn_energy::Float64, nnn_energy::Float64, adsorption_energy::Float64, lattice_type::Symbol=:square)
    energies, configurations = exact_enumeration(L, L, N, lattice_type, nn_energy, nnn_energy, adsorption_energy)
    
    minimum_energy = minimum(energies)
    energies = energies .- minimum_energy

    BoltzmannConstant = 8.617_333_262e-5
    temperatures = range(T_min, T_max, length=num_points)
    internal_energies = [sum(energy * exp(-energy / (BoltzmannConstant * T)) for energy in energies) / sum(exp(-energy / (BoltzmannConstant * T)) for energy in energies) for T in temperatures]
    heat_capacity = [sum((energy - U)^2 * exp(-energy / (BoltzmannConstant * T)) for energy in energies) / (BoltzmannConstant * T^2 * sum(exp(-energy / (BoltzmannConstant * T)) for energy in energies)) for (T, U) in zip(temperatures, internal_energies)]

    internal_energies = internal_energies .+ minimum_energy

    println("Temperature (K) | Internal Energy (eV) | Heat Capacity (eV/K)")
    for (T, U, C) in zip(temperatures, internal_energies, heat_capacity)
        println("$T $U $C")
    end
end

# Example Usage
L = 4
M = 4
N = 2
T_min = 1.0
T_max = 100.0
num_points = 100
nn_energy = -0.01
nnn_energy = nn_energy / 4
adsorption_energy = 4 * nn_energy
# TODO: Define the lattice constant here

compute_internal_energy_versus_temperature(L, N, T_min, T_max, num_points, nn_energy, nnn_energy, adsorption_energy)

Temperature (K) | Internal Energy (eV) | Heat Capacity (eV/K)
1.0 -0.09 1.0385161321682132e-38
2.0 -0.09 2.058363768587058e-20
3.0 -0.08999999999999811 1.824512923132884e-14
4.0 -0.08999999999733195 1.4521161074878097e-11
5.0 -0.08999999979181746 7.264482518124503e-10
6.0 -0.08999999616896903 9.318117856063588e-9
7.0 -0.08999996902076408 5.5679366427903604e-8
8.0 -0.08999984988523949 2.0811591421287678e-7
9.0 -0.08999948263250401 5.716211617630698e-7
10.0 -0.089998595405126 1.2685623593190118e-6
11.0 -0.08999679579569829 2.413668796251124e-6
12.0 -0.0899935893140408 4.093687371760758e-6
13.0 -0.08998841385318762 6.355979787806911e-6
14.0 -0.08998068108452491 9.205784204274062e-6
15.0 -0.08996981718716394 1.260983719967114e-5
16.0 -0.08995529788714696 1.650371664053168e-5
17.0 -0.08993667523335777 2.0800714530959192e-5
18.0 -0.08991359539980638 2.5400710932820345e-5
19.0 -0.089885808019732 3.0198125712681585e-5
20.0 -0.08985316823686669 3.50884856988174e-5
21.0 -0.0898156329533797 3.997